<a href="https://colab.research.google.com/github/val93s/Machine_learning/blob/main/Copy_of_11_4_3_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#11.4.3 Activity

#**Breast Cancer Detection**

According to the [American Cancer Society (2022)](https://www.cancer.org/cancer/breast-cancer/about/how-common-is-breast-cancer.html),
 breast cancer is the most common cancer in American women, except for skin cancers. The average risk of a woman in the United States developing breast cancer sometime in her life is about 13%. This means there is a 1 in 8 chance she will develop breast cancer.

Mammograms are used to detect breast cancer—hopefully at an early stage.  However, many masses that appear on a mammogram are not actually cancer.  Developing a machine learning model to predict whether a tumor is benign or cancerous would be helpful for physicians as they guide and treat patients. 

In this activity, we will use cross-validation to evaluate our model's predictive ability.



##Step 1: Download and save the `cancer.csv` dataset from the class materials  

Make a note of where you saved the file on your computer.

##Step 2: Upload the `cancer.csv` dataset by running the following code block 

When prompted, navigate to and select the `cancer.csv` dataset where you saved it on your computer.

In [ ]:
#Step 2

from google.colab import files
cancer = files.upload()

Saving cancer.csv to cancer (1).csv


##Step 3: Import necessary packages

```
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np

```

In [ ]:
#Step 3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate




## Step 4: Create a Pandas DataFrame from the CSV file
* Name the DataFrame `cancer`.
* Print the first five observations of `cancer`.  Note the kinds of data it contains.

In [ ]:
#Step 4
cancer = pd.read_csv('cancer.csv')
cancer.head()
cancer.describe()




,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,565.000000,569.000000,569.000000,569.000000,564.000000,563.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096336,0.104341,0.088799,0.048919,0.181233,0.062792
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014029,0.052813,0.079720,0.038803,0.027453,0.007050
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161975,0.057750
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095860,0.092630,0.061540,0.033500,0.179250,0.061550
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440


##Step 5: Convert the variable `diagnosis` into a numeric data type  
* There are many way to accomplish this, but you may choose to work with the example shown below.  

```
cancer.loc[cancer['diagnosis'] == 'M', 'cancer_present'] = 1
cancer.loc[cancer['diagnosis'] == 'B', 'cancer_present'] = 0

```
* Name the result `cancer_present` and code malignant tumors with a `1` and benign tumors with a `0`.






In [ ]:
#Step 5
cancer.loc[cancer['diagnosis'] == 'M', 'cancer_present'] = 1
cancer.loc[cancer['diagnosis'] == 'B', 'cancer_present'] = 0


##Step 6: Split the data into the target variable and the feature of interest
* You want to predict whether a tumor is benign or malignant (`cancer_present`) using the mean tumor perimeter measure (`perimeter_mean`).
* Select the all of the features of the `cancer` DataFrame **except** `id`, `diagnosis`, and `cancer_present`, and name the resulting DataFrame `X`.
* Select the column `cancer_present` from the `cancer` DataFrame and name it `y`. Make sure `y` is also a DataFrame and not a Series.

In [ ]:
#Step 6
y = cancer[['cancer_present']]

X = cancer[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean','compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]


##Step 7: Split the data into a training dataset and a test dataset
* Use `train_test_split` from `sklearn.model_selection`.
* Name the *X* training set `X_train` and the *y* training set `y_train`.
* Name the *X* test set `X_test` and the *y* test set `y_test`.
* Set the `test_size = 0.25` and `random_state = 42`. 






In [ ]:
#Step 7
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.25, random_state=42)



##Step 8: Build a pipeline that will impute and standardize the data and fit a logistic regression model
* The first step is `SimpleImputer(missing_values=np.nan, strategy='mean'))`.
* The second step is `StandardScaler()`.
* And the third step is `LogisticRegression(random_state=0)`.
* Name the pipeline `pipe`.
* Fit the pipeline to `X_train` and `Y_train`.





In [ ]:
#Step 8
pipe_std = Pipeline([('imp_mean', SimpleImputer(missing_values=np.nan, strategy='mean')),
                    ('scaler', StandardScaler()),
                    ('log_reg', LogisticRegression(random_state=0))])





##Step 9: Evaluate the pipeline using 5-fold cross-validation  
* Calculate and print the accuracy of each of the five models using `scores = cross_val_score(pipe, X_train, y_train, cv=5)`.
* Calculate and print the mean and SD of the accuracy measures returned from cross-validation.





In [ ]:
#Step 9
scores = cross_val_score(pipe, X_train, y_train, cv=5)

print(scores)
print(scores.mean())
print(scores.std())





NameError: ignored

##Step 10: Print the mean accuracy for different values of `k`  
* Run the following code block to calculate mean accuracy for *k=2* to *k=25*.
* The code below will also print a graph of the mean accuracy for each `k`.
* In this specific example, does increasing `k` seem to increase the model accuracy? Or is accuracy similarly high for all values of `k` (ignoring small fluctuations)?





In [ ]:
#Step 10

k = list(range(2,26))
mean_accuracy = []

for i in k:
  scores = cross_val_score(pipe, X_train, y_train, cv=i)
  mean_accuracy.append(scores.mean())


k_df = pd.DataFrame(k)
k_df.rename({0:'k'}, axis=1, inplace=True)

mean_accuracy_df = pd.DataFrame(mean_accuracy)*100
mean_accuracy_df.rename({0:'mean accuracy'}, axis=1, inplace=True)

to_plot = pd.concat([k_df, mean_accuracy_df], axis=1)

plt.plot(to_plot['k'], to_plot['mean accuracy'])
plt.xlabel('k')
plt.ylabel('Mean accuracy %')
plt.show()


###Answer:



##Step 11: Fit the model to the test set
* Calculate and print the test set accuracy.
* Is the accuracy you calculated using 5-fold cross-validation a good estimate of the testing accuracy?




In [ ]:
#Step 11
test_accuracy = pipe.score(X_test, y_test)
print('The testing accuracy is', test_accuracy)



###Answer: